## **Dependencies**

In [2]:
#pip install "u8darts[notorch]" lightgbm

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.7 MB 812.7 kB/s eta 0:02:06
   ---------------------------------------- 0.2/101.7 MB 1.1 MB/s eta 0:01:29
   ---------------------------------------- 0.2/101.7 MB 1.1 MB/s eta 0:01:32
   ---------------------------------------- 0.3/101.7 MB 1.4 MB/s eta 0:01:12
   ---------------------------------------- 0.5/101.7 MB 2.1 MB/s eta 0:00:48
   ---------------------------------------- 0.7/101.7 MB 2.3 MB/s eta 0:00:45
   ---------------------------------------- 0.9/101.7 MB 2.4 MB/s eta 0:00:43
   ---------------------------------------- 1.0/101.7 MB 2.4 MB/s eta 0:00:43
   ---------------------------------------- 1.0/101.7 MB 2.5 MB/s eta 0:00:40
   ---------------------------------------- 1.1/101.7 MB 2.2 MB/s eta 0:00:46
    --------------------------------------- 1.3/101.7 MB 2.3 MB/s eta 0:00:4

In [1]:
import pandas as pd
import numpy as np
import darts
from darts import TimeSeries
import matplotlib.pyplot as plt

## **Dataset**

In [2]:
df_sales_0 = pd.read_csv(r"C:\Users\HAI\Downloads\Phase 0 - Sales.csv") # Phase 0 sales
df_price_0 = pd.read_csv(r"C:\Users\HAI\Downloads\Phase 0 - Price.csv") # Phase 0 Price
df_sales_0.set_index(["Client", "Warehouse", "Product"], inplace=True)
df_price_0.set_index(["Client", "Warehouse", "Product"], inplace=True)

In [3]:
df_sales_1 = pd.read_csv(r"C:\Users\HAI\Downloads\Phase 1 - Sales.csv") # Phase 1 sales
df_price_1 = pd.read_csv(r"C:\Users\HAI\Downloads\Phase 1 - Price.csv") # Phase 1 Price
df_sales_1.set_index(["Client", "Warehouse", "Product"], inplace=True)
df_price_1.set_index(["Client", "Warehouse", "Product"], inplace=True)

In [4]:
# Concatenate the two dataframes
df_sales = pd.merge(df_sales_0, df_sales_1, how='outer', on=["Client", "Warehouse", "Product"])
df_price = pd.merge(df_price_0, df_price_1, how='outer', on=["Client", "Warehouse", "Product"])

In [5]:
df_sales.reset_index(inplace=True)

In [6]:
df_price.reset_index(inplace=True)

### **Merge data**

In [9]:
# Function for converting time series to long form
def make_long_format(df, value_name):
  long_format_df = pd.melt(
    df,
    id_vars=['Client', 'Warehouse', 'Product'],
    var_name='Date',
    value_name=value_name,
  )
  long_format_df['Date'] = pd.to_datetime(long_format_df['Date'])
  return long_format_df

In [10]:
long_format_sales=make_long_format(df_sales, 'Sales')
long_format_price=make_long_format(df_price, 'Price')

## **1. Data preprocessing**

#### Need to handle missing values: Analyzing trends in missing data
- **Price**:

  + Missing: no transactions --> Handle: fill some values (...) due to sales == 0 -> Price = Nan ------ DONE


In [11]:
# Sort product with time order
long_format_price.sort_values(by=['Product', 'Warehouse'], inplace =True)
long_format_price

,Client,Warehouse,Product,Date,Price
946,42,45,36,2020-07-06,NaN
2783,42,45,36,2020-07-13,NaN
4620,42,45,36,2020-07-20,NaN
6457,42,45,36,2020-07-27,NaN
8294,42,45,36,2020-08-03,NaN
...,...,...,...,...,...
328253,43,114,14322,2023-12-04,59.949997
330090,43,114,14322,2023-12-11,59.950000
331927,43,114,14322,2023-12-18,59.949997
333764,43,114,14322,2023-12-25,59.950000


In [12]:
# Filling nan in the first week with backfilling
long_format_price.loc[(long_format_price['Date'] == '2020-07-06') & (long_format_price['Price'].isnull()), 'Price'] = long_format_price.bfill()
long_format_price

,Client,Warehouse,Product,Date,Price
946,42,45,36,2020-07-06,18.820000
2783,42,45,36,2020-07-13,NaN
4620,42,45,36,2020-07-20,NaN
6457,42,45,36,2020-07-27,NaN
8294,42,45,36,2020-08-03,NaN
...,...,...,...,...,...
328253,43,114,14322,2023-12-04,59.949997
330090,43,114,14322,2023-12-11,59.950000
331927,43,114,14322,2023-12-18,59.949997
333764,43,114,14322,2023-12-25,59.950000


In [ ]:
# Filling Nan values of long_format_price with foward fill
long_format_price['Price'] = long_format_price['Price'].fillna(method='ffill')


In [14]:
train_df = pd.merge(long_format_sales, long_format_price, on=['Client', 'Warehouse', 'Product', 'Date'], how='outer')
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df

,Client,Warehouse,Product,Date,Sales,Price
0,41,2,1178,2020-07-06,828.0,14.970000
1,41,2,1222,2020-07-06,3515.0,18.842463
2,41,2,2325,2020-07-06,1157.0,23.941650
3,41,2,6384,2020-07-06,0.0,179.990000
4,41,2,6743,2020-07-06,632.0,16.942880
...,...,...,...,...,...,...
336166,46,318,13485,2024-01-01,0.0,30.000000
336167,46,318,13582,2024-01-01,67.0,23.105225
336168,46,318,13691,2024-01-01,2.0,26.495000
336169,46,318,13946,2024-01-01,0.0,30.990000


## **EDA**

## **2. Feature Engineering**


### *Rolling mean feature* <br>

In [15]:
def Rolling_mean(col, window):
    Rolling_mean= pd.DataFrame(train_df.groupby(['Product', 'Client', 'Warehouse'])[col].rolling(window).mean())
    Rolling_mean.rename(columns={col:"Rolling_mean_"+str(col)+str(window)}, inplace=True)
    Rolling_mean.reset_index(inplace=True)
    Rolling_mean.set_index(['level_3'], inplace=True)
    Rolling_mean.drop(columns=['Product', 'Client', 'Warehouse'], inplace=True)

    return Rolling_mean

In [16]:
Rolling_mean = Rolling_mean('Sales', 4)
# Merging back
train_df = pd.concat([train_df, Rolling_mean], axis=1)
train_df

,Client,Warehouse,Product,Date,Sales,Price,Rolling_mean_Sales4
0,41,2,1178,2020-07-06,828.0,14.970000,NaN
1,41,2,1222,2020-07-06,3515.0,18.842463,NaN
2,41,2,2325,2020-07-06,1157.0,23.941650,NaN
3,41,2,6384,2020-07-06,0.0,179.990000,NaN
4,41,2,6743,2020-07-06,632.0,16.942880,NaN
...,...,...,...,...,...,...,...
336166,46,318,13485,2024-01-01,0.0,30.000000,1.00
336167,46,318,13582,2024-01-01,67.0,23.105225,43.50
336168,46,318,13691,2024-01-01,2.0,26.495000,1.00
336169,46,318,13946,2024-01-01,0.0,30.990000,0.25


In [17]:
def Rolling_mean(col, window):
    Rolling_mean= pd.DataFrame(train_df.groupby(['Product', 'Client', 'Warehouse'])[col].rolling(window).mean())
    Rolling_mean.rename(columns={col:"Rolling_mean_"+str(col)+str(window)}, inplace=True)
    Rolling_mean.reset_index(inplace=True)
    Rolling_mean.set_index(['level_3'], inplace=True)
    Rolling_mean.drop(columns=['Product', 'Client', 'Warehouse'], inplace=True)

    return Rolling_mean

In [18]:
Rolling_mean_6 = Rolling_mean('Sales', 6)

In [19]:
# Merging back
train_df = pd.concat([train_df, Rolling_mean_6], axis=1)
train_df

,Client,Warehouse,Product,Date,Sales,Price,Rolling_mean_Sales4,Rolling_mean_Sales6
0,41,2,1178,2020-07-06,828.0,14.970000,NaN,NaN
1,41,2,1222,2020-07-06,3515.0,18.842463,NaN,NaN
2,41,2,2325,2020-07-06,1157.0,23.941650,NaN,NaN
3,41,2,6384,2020-07-06,0.0,179.990000,NaN,NaN
4,41,2,6743,2020-07-06,632.0,16.942880,NaN,NaN
...,...,...,...,...,...,...,...,...
336166,46,318,13485,2024-01-01,0.0,30.000000,1.00,8.500000
336167,46,318,13582,2024-01-01,67.0,23.105225,43.50,37.500000
336168,46,318,13691,2024-01-01,2.0,26.495000,1.00,2.500000
336169,46,318,13946,2024-01-01,0.0,30.990000,0.25,0.500000


### *Price Rolling mean feature*


In [20]:
def Rolling_mean(col, window):
    Rolling_mean= pd.DataFrame(train_df.groupby(['Product', 'Client', 'Warehouse'])[col].rolling(window).mean())
    Rolling_mean.rename(columns={col:"Rolling_mean_"+str(col)+str(window)}, inplace=True)
    Rolling_mean.reset_index(inplace=True)
    Rolling_mean.set_index(['level_3'], inplace=True)
    Rolling_mean.drop(columns=['Product', 'Client', 'Warehouse'], inplace=True)

    return Rolling_mean

In [21]:
mean_price = Rolling_mean('Price', 4)
mean_price_6 = Rolling_mean('Price', 6)
train_df = pd.concat([train_df, mean_price, mean_price_6], axis=1)
train_df

,Client,Warehouse,Product,Date,Sales,Price,Rolling_mean_Sales4,Rolling_mean_Sales6,Rolling_mean_Price4,Rolling_mean_Price6
0,41,2,1178,2020-07-06,828.0,14.970000,NaN,NaN,NaN,NaN
1,41,2,1222,2020-07-06,3515.0,18.842463,NaN,NaN,NaN,NaN
2,41,2,2325,2020-07-06,1157.0,23.941650,NaN,NaN,NaN,NaN
3,41,2,6384,2020-07-06,0.0,179.990000,NaN,NaN,NaN,NaN
4,41,2,6743,2020-07-06,632.0,16.942880,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
336166,46,318,13485,2024-01-01,0.0,30.000000,1.00,8.500000,30.000000,28.996667
336167,46,318,13582,2024-01-01,67.0,23.105225,43.50,37.500000,23.030748,23.017166
336168,46,318,13691,2024-01-01,2.0,26.495000,1.00,2.500000,19.871250,30.674226
336169,46,318,13946,2024-01-01,0.0,30.990000,0.25,0.500000,32.515000,32.515000


## **3. Models**

### LightGBM


In [22]:
from darts.models import LightGBMModel

In [23]:
# Datetime features
from darts.utils.timeseries_generation import datetime_attribute_timeseries

full_time_period = pd.date_range(start='2020-07-06', end='2024-04-01', freq='W-MON')

year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
time_cov = year.stack(month).stack(weekofyear)
time_cov = time_cov.astype(np.float32)


#### **Prepare Input**

In [24]:
from tqdm import tqdm
warehouse_list = train_df['Warehouse'].unique()
product_list = train_df['Product'].unique()
client_list = train_df['Client'].unique()
display(warehouse_list)
display(product_list)

array([  2,  14,  22,  23,  28,  35,  38,  54,  70,  76,  78,  84,  88,
        89, 110, 111, 115, 125, 127, 128, 132, 133, 143, 152, 159, 163,
       165, 171, 172, 181, 183, 184, 185, 197, 200, 201, 206, 209, 210,
       211, 212, 213, 218, 220, 227, 228, 231, 233, 246, 253, 255, 263,
       269, 270, 271, 282, 283, 288, 300, 301, 330,  34,  45,  87,  90,
       102, 153, 156, 174, 229, 232, 245, 114, 268, 319,  26, 337,  24,
       107, 112, 134, 160, 195, 318], dtype=int64)

array([1178, 1222, 2325, ..., 3742, 6019, 9969], dtype=int64)

In [25]:
# Sales data
product_TS_dict = {}

for product in tqdm(product_list):
  df_product = train_df.loc[train_df['Product'] == product]

  list_of_TS_product = TimeSeries.from_group_dataframe(
                                df_product,
                                time_col="Date",
                                group_cols=["Warehouse", "Product"],
                                static_cols="Client",
                                value_cols=["Sales"]
                                )
  for ts in list_of_TS_product:
            ts = ts.astype(np.float32)

  list_of_TS_product = sorted(list_of_TS_product, key=lambda ts: int(ts.static_covariates_values()[0,0]))
  product_TS_dict[product] = list_of_TS_product

100%|██████████| 1435/1435 [00:12<00:00, 116.62it/s]


In [26]:
# Past covariate dict
past_covariate_dict = {}
feature_df = train_df.drop(columns=['Sales', 'Client'])
for product in tqdm(product_list):
    df_product = feature_df.loc[feature_df['Product'] == product]

    list_of_TS_past_covariate = TimeSeries.from_group_dataframe(
                                df_product,
                                time_col="Date",
                                group_cols=["Warehouse", "Product"]
                                )

    for ts in list_of_TS_past_covariate:
        ts = ts.astype(np.float32)

    past_covariate_dict[product] = list_of_TS_past_covariate

  0%|          | 0/1435 [00:00<?, ?it/s]

100%|██████████| 1435/1435 [00:11<00:00, 120.17it/s]


In [27]:
# Future covariate dict
future_covariates_dict = {}

for key in tqdm(past_covariate_dict):
    covariates_future = [time_cov for i in range(0, len(past_covariate_dict[key]))]
    future_covariates_dict[key] = covariates_future

100%|██████████| 1435/1435 [00:00<00:00, 709516.24it/s]


In [28]:
# Training function
def lgbm_predictions(model_params, val_df_size = None):
    last_train_date = pd.to_datetime(train_df.Date.max())
    l_train_date = last_train_date - np.timedelta64(val_df_size, 'W')

    submission_kaggle_list = []
    cnt = 1

    for params in model_params:
        LGBM_Models_Submission = {}
        display("Training...")

        # Fit Model
        print(f'Start fit model {cnt}')
        for product in tqdm(product_list):
            sales_product = product_TS_dict[product]
            past_cov_product = past_covariate_dict[product]
            future_product = future_covariates_dict[product]
            # training_data: represents the number of sales in the training sample minus the sales for the val
            training_data = [ts[:183-val_df_size] for ts in sales_product]
            covariate_data = [ts[:183-val_df_size] for ts in past_cov_product]
            validation_series = [ts[:-val_df_size] for ts in sales_product]
            validation_covariates = [ts[:-val_df_size] for ts in past_cov_product]
            # TCN_covariates: represents the future covariates associated with the target product family
            #TCN_covariates =[ts[:170-val_df_size] for ts in future_product]
            TCN_valid = [ts[:-(13+val_df_size)] for ts in future_product]
            # train_sliced: represents the number of sales associated with the target product family.
            # slice_intersect: function that you can see used simply ensures that the components span the same time interval.
            # In the case of different time intervals an error message will appear if we try to combine them.
            train_sliced = [training_data[i].slice_intersect(future_product[i]) for i in range(0,len(training_data))]
            valid_sliced = [validation_series[i].slice_intersect(future_product[i]) for i in range(0,len(validation_series))]

            LGBM_Model_Submission = LightGBMModel(lags = params["lags"],
                                                  lags_future_covariates = params["lags_future_covariates"],
                                                  lags_past_covariates = params["lags_past_covariates"],
                                                  output_chunk_length=13,
                                                  min_child_samples=5,
                                                  early_stopping_round = 5,
                                                  verbose = -1,
                                                  random_state=42)


            LGBM_Model_Submission.fit(series=train_sliced, future_covariates=future_product, past_covariates=covariate_data, val_series=valid_sliced, val_past_covariates=validation_covariates, val_future_covariates=TCN_valid)

            LGBM_Models_Submission[product] = LGBM_Model_Submission

        display("Predictions...")
        LGBM_Forecasts_Families_Submission = {}

        # Predict
        print(f'Start predict model {cnt}')
        for product in tqdm(product_list):
            sales_product = product_TS_dict[product]
            past_cov_product = past_covariate_dict[product]
            future_product = future_covariates_dict[product]
            training_data = [ts[:183] for ts in sales_product]
            LGBM_covariates = [ts[:183] for ts in past_cov_product]
            LGBM_future_covariates = [ts[:196] for ts in future_product]
            
            train_sliced = [training_data[i].slice_intersect(LGBM_future_covariates[i]) for i in range(0,len(training_data))]

            forecast_LGBM = LGBM_Models_Submission[product].predict(
                                                  n= 13,
                                                  series=train_sliced,
                                                  future_covariates=LGBM_future_covariates,
                                                  past_covariates= LGBM_covariates
                                                 )

            LGBM_Forecasts_Families_Submission[product] = forecast_LGBM

        # 0 forecasting function
        for product in tqdm(LGBM_Forecasts_Families_Submission):
            for n in range(0, len(LGBM_Forecasts_Families_Submission[product])):
                if (product_TS_dict[product][n].univariate_values()[-3:] == 0).all():
                    LGBM_Forecasts_Families_Submission[product][n] = LGBM_Forecasts_Families_Submission[product][n].map(lambda x: x * 0)

        print(f'Preparing submission {cnt}')
        # Append the predictions of each product
        a = pd.DataFrame()
        for product in tqdm(LGBM_Forecasts_Families_Submission):
            for n in range(0, len(LGBM_Forecasts_Families_Submission[product])):

                temp_df = LGBM_Forecasts_Families_Submission[product][n].pd_dataframe()
                temp_df.reset_index(inplace=True)
                temp_df['Product'] = product
                dataframe=product_TS_dict[product][n].static_covariates
                temp_df['Warehouse'] = dataframe['Warehouse'].values[0]
                temp = train_df[train_df['Product']== product][train_df['Warehouse']== dataframe['Warehouse'].values[0]]
                temp.drop(columns=['Price'], inplace=True)
                temp = pd.concat([temp, temp_df], axis=0, ignore_index=True)
                temp.ffill(inplace=True)
                a = pd.concat([a, temp], axis=0, ignore_index=True)
        # Submission format
        a['Date'] = pd.to_datetime(a['Date'])
        b=pd.pivot_table(a, values='Sales', index=['Client', 'Warehouse', 'Product'], columns='Date')
        b.clip(lower=0, inplace=True)
        b = b.iloc[:, -13:]

    return b

In [29]:
# Results
import warnings 
warnings.filterwarnings("ignore")
model_params = [{"lags" : 6, "lags_future_covariates" : [1,2,3,4,5,6], "lags_past_covariates" : 6}]
submission = lgbm_predictions(model_params, val_df_size = 13)

'Training...'

Start fit model 1


100%|██████████| 1435/1435 [12:23<00:00,  1.93it/s]


'Predictions...'

Start predict model 1


100%|██████████| 1435/1435 [00:00<00:00, 1548.83it/s]


Preparing submission 1


100%|██████████| 1435/1435 [00:21<00:00, 66.01it/s]


In [30]:
# Rounding values of final_result_df
import math
submission = submission.applymap(lambda x: math.ceil(x))
submission

Date                      2024-01-08  2024-01-15  2024-01-22  2024-01-29  \
Client Warehouse Product                                                   
41.0   2.0       1178            712         675         818         779   
                 1222           1515        1695        1957        2052   
                 2325            505         537         529         583   
                 6384              3           1           2           1   
                 6743            311         367         318         342   
...                              ...         ...         ...         ...   
46.0   318.0     13485             5           8           6           5   
                 13582            57          58          50          78   
                 13691             4           1           5           5   
                 13946             0           1           5          10   
                 14294             2           4           7           8   

Date                      2024-02-05  2024-02-12  2024-02-19  2024-02-26  \
Client Warehouse Product                                                   
41.0   2.0       1178            690         682         873         791   
                 1222           2181        1641        1345        1215   
                 2325            436         352         480         435   
                 6384              6           5           1           3   
                 6743            343         337         414         354   
...                              ...         ...         ...         ...   
46.0   318.0     13485            17          34           1           9   
                 13582            66          82          81          88   
                 13691             3           4           5           6   
                 13946             0           0           0           0   
                 14294            11          11          11          10   

Date                      2024-03-04  2024-03-11  2024-03-18  2024-03-25  \
Client Warehouse Product                                                   
41.0   2.0       1178            830         675         621         801   
                 1222           2062        2516        1772        2337   
                 2325            458         297         369         353   
                 6384              3           2           3           2   
                 6743            416         404         355         332   
...                              ...         ...         ...         ...   
46.0   318.0     13485            11          65          73          46   
                 13582            59          43         232         316   
                 13691             5           4           1           2   
                 13946             1           1           0           2   
                 14294             7           5           9           5   

Date                      2024-04-01  
Client Warehouse Product              
41.0   2.0       1178            980  
                 1222           2209  
                 2325            406  
                 6384              1  
                 6743            337  
...                              ...  
46.0   318.0     13485            44  
                 13582            79  
                 13691             2  
                 13946             0  
                 14294             6  

[1837 rows x 13 columns]

In [31]:
submission.to_csv("phase_2_13216_15053.csv")